In [1]:
import igl
import meshplot as mp
import numpy as np
from Modules.Mesh import Mesh, Patch
from Modules.RotationMatrix import RotationMatrix
from Modules.PatchCollector import PatchCollector

In [2]:
myMesh = Mesh.readFile('Object/example_object.obj')
myCollector = PatchCollector(myMesh)

In [3]:
bcs = igl.barycenter(myMesh.v, myMesh.f)
faces = np.arange(len(myMesh.f))
cornerNormals = myMesh.getFaceNormals()
end = bcs + cornerNormals
p = mp.plot(myMesh.v, myMesh.f, shading={"wireframe": True})
p.add_lines(bcs, end, shading={"line_color": "green"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, -1.6…

1

In [4]:
AMOUNT_OF_COPIES = 10
RANDOM_NUMBER = np.random.randint(0, len(myMesh.f))
SHIFT = np.array([4, 0, 0])
patch = myCollector.selectPaperPatch(RANDOM_NUMBER)
patches = []
rotations = []
for i in range(AMOUNT_OF_COPIES):
    new_patch = patch.copy()
    rotation = RotationMatrix.getRandomRotationMatrix()
    new_patch.rotate(rotation)
    rotationMatrix = new_patch.getPaperRotationMatrix()
    new_patch.translate(SHIFT*i)
    patches.append(new_patch)
    rotations.append(rotationMatrix)
    # new_patch.rotate(rotationMatrix.matrix)

p = Mesh.mpViewMeshes(*patches)
for i, rotation in enumerate(rotations):
    start1 = rotation.cj + SHIFT*i
    start2 = rotation.ci + SHIFT*i
    start3 = np.tile(rotation.ci + SHIFT*i, (rotation.cj.shape[0], 1))
    end1 = rotation.njprime * rotation.muj[:, None]
    end2 = np.sum(end1, axis=0)
    end3 = rotation.dcs
    p.add_lines(start1, start1 + end1, shading={"line_color": "red"})
    p.add_lines(start2, start2 + end2, shading={"line_color": "green"})
    p.add_lines(start3, start3 + end3, shading={"line_color": "blue"})
    

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(47.880724…

In [5]:
fi = 10
patch = myCollector.selectPaperPatch(fi)
patch.translate(-1*patch.getPCCenter()).resize(patch.getPCSize() / np.max(patch.getPCBoundingBox()))
rotationMatrix = patch.getPaperRotationMatrix()

VISUALIZATION_OFFSET = 2
patch = myCollector.selectPaperPatch(fi)
patches = []
rotations = []
for i in range(10):
    new_patch = patch.copy()
    rotation = RotationMatrix.getRandomRotationMatrix()
    new_patch.translate(-patch.getPCCenter()).resize(patch.getPCSize() / np.max(patch.getPCBoundingBox())).rotate(rotation).translate(np.array([VISUALIZATION_OFFSET*i, 0, 0]))
    rotationMatrix = new_patch.getPaperRotationMatrix()
    patches.append(new_patch)
    rotations.append(rotationMatrix)

plot = Mesh.mpViewMeshes(*patches)
for i, rotationMatrix in enumerate(rotations):
    start = rotationMatrix.ci.reshape((3,))
    end = start + np.sum(rotationMatrix.njprime * rotationMatrix.muj[:, None], axis=0) / 10
    plot.add_lines(start, end, shading={"line_color": "red"})
    # print(np.all(np.linalg.eigvals(rotationMatrix.Ti) > 0))
    eigTest = np.linalg.eig(rotationMatrix.Ti)
    identity = np.identity(rotationMatrix.matrix.shape[0])
    Ti_test = np.zeros((3, 3))
    for i in range(3):
        eigenvalue = eigTest[0][i]
        eigenvector = eigTest[1][:, i][:, None]
        result = eigenvalue * eigenvector * eigenvector.T
        Ti_test += result
    # print(rotationMatrix.matrix * rotationMatrix.matrix.T)


TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_9fba081e-d2b7-4744-92ba-c6a22010ad6a'.

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(9.0111930…

In [6]:
random_index = np.random.randint(0, len(myCollector.mesh.f))
testPatch = myCollector.selectPaperPatch(random_index)
print(f"Random face index {random_index} has been chosen for this test!")
testPatch.v += (np.random.rand(*testPatch.v.shape) * 2 - 1) / 10
testPatch.translate(-testPatch.getPCCenter())
testPatch.resize(1)
testMatrix = testPatch.getPaperRotationMatrix().matrix
print(f"Test Rotation Matrix: {testMatrix}\nMatrix is Rotation Matrix: {np.sum(np.linalg.norm(testMatrix, axis=1))==3}, cause sum is {np.sum(np.linalg.norm(testMatrix, axis=1))}")
testPatch.rotate(testMatrix)
AMOUNT_OF_COPIES = 10
copiedPatches = [testPatch]
matrices = [testMatrix]
for i in range(AMOUNT_OF_COPIES):
    copyPatch = copiedPatches[-1].copy()
    matrix = copyPatch.getPaperRotationMatrix().matrix
    print(f"Position: {copyPatch.getPCCenter()}\nSize: {copyPatch.getPCSize()}\nRotation Matrix {i}: {matrix}\nMatrix is Rotation Matrix: {np.sum(np.linalg.norm(matrix, axis=1))==3}, cause sum is {np.sum(np.linalg.norm(matrix, axis=1))}")
    copyPatch.alignPatch()
    matrices.append(matrix)
    copiedPatches.append(copyPatch)

plot = Mesh.mpViewMeshes(*copiedPatches)
truth_table = np.full((AMOUNT_OF_COPIES, AMOUNT_OF_COPIES), False)
for i in range(AMOUNT_OF_COPIES):
    for j in range(AMOUNT_OF_COPIES):
        truth_table[i][j] = np.allclose(matrices[i], matrices[j])
print(truth_table)
for i in range(AMOUNT_OF_COPIES + 1):
    plot.add_lines(copiedPatches[i].getPCCenter(), copiedPatches[i].getPCCenter() + matrices[i].T[:, 0] * 4)
    plot.add_lines(copiedPatches[i].getPCCenter(), copiedPatches[i].getPCCenter() + matrices[i].T[:, 1] * 4)
    plot.add_lines(copiedPatches[i].getPCCenter(), copiedPatches[i].getPCCenter() + matrices[i].T[:, 2] * 4)

TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_6f13be3f-0f6a-4069-aca9-f12a2a766f96'.

Random face index 34032 has been chosen for this test!
Test Rotation Matrix: [[ 0.02350976  0.99968214  0.00910583]
 [ 0.91127626 -0.02517462  0.41102531]
 [ 0.4111239  -0.00136518 -0.91157845]]
Matrix is Rotation Matrix: False, cause sum is 3.000000000000001
Position: [ 3.05311332e-16 -4.53341068e-16  9.96116769e-16]
Size: 1.0000000000000002
Rotation Matrix 0: [[ 1.00000000e+00 -5.07665263e-17  3.40105805e-17]
 [ 5.07665263e-17  1.00000000e+00 -7.77156117e-16]
 [-3.40105805e-17  7.77156117e-16  1.00000000e+00]]
Matrix is Rotation Matrix: False, cause sum is 2.9999999999999996
Position: [ 2.91433544e-16 -4.50257116e-16  1.00228468e-15]
Size: 1.0
Rotation Matrix 1: [[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00 -3.33066907e-16]
 [ 0.00000000e+00  1.11022302e-16  1.00000000e+00]]
Matrix is Rotation Matrix: True, cause sum is 3.0
Position: [ 2.77555756e-16 -4.44089210e-16  1.01462049e-15]
Size: 1.0
Rotation Matrix 2: [[ 1.00000000e+00  0.00000000e+00 

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.069328…

[[ True False False False False False False False False False]
 [False  True  True False  True  True  True  True  True  True]
 [False  True  True False  True  True  True  True  True  True]
 [False False False  True False False False False False False]
 [False  True  True False  True  True  True  True  True  True]
 [False  True  True False  True  True  True  True  True  True]
 [False  True  True False  True  True  True  True  True  True]
 [False  True  True False  True  True  True  True  True  True]
 [False  True  True False  True  True  True  True  True  True]
 [False  True  True False  True  True  True  True  True  True]]


In [7]:
fi = 10
patch = myCollector.selectPaperPatch(fi)
patch.translate(-1*patch.getPCCenter()).resize(patch.getPCSize() / np.max(patch.getPCBoundingBox()))
rotationMatrix = patch.getPaperRotationMatrix()

VISUALIZATION_OFFSET = 2
patch = myCollector.selectPaperPatch(fi)
patches = []
rotations = []
random_rotations = []
for i in range(10):
    new_patch = patch.copy()
    rotation = RotationMatrix.getRandomRotationMatrix()
    new_patch.translate(-patch.getPCCenter()).resize(patch.getPCSize() / np.max(patch.getPCBoundingBox())).rotate(rotation.T).translate(np.array([VISUALIZATION_OFFSET*i, 0, 0]))
    rotationMatrix = new_patch.getPaperRotationMatrix()
    patches.append(new_patch)
    rotations.append(rotationMatrix)
    random_rotations.append(rotation)

plot = Mesh.mpViewMeshes(*patches)
for i, rotationMatrix in enumerate(rotations):
    center = patches[i].getPCCenter()
    end1 = center + np.identity(3).dot(random_rotations[i][0])
    end2 = center + np.identity(3).dot(random_rotations[i][1])
    end3 = center + np.identity(3).dot(random_rotations[i][2])
    v0 = center + rotationMatrix.eig[1].T[0]
    v1 = center + rotationMatrix.eig[1].T[1]
    v2 = center + rotationMatrix.eig[1].T[2]
    plot.add_lines(center, end1, shading={"line_color": "red"})
    plot.add_lines(center, end2, shading={"line_color": "green"})
    plot.add_lines(center, end3, shading={"line_color": "blue"})
    plot.add_lines(center, v0, shading={"line_color": "yellow"})
    plot.add_lines(center, v1, shading={"line_color": "cyan"})
    plot.add_lines(center, v2, shading={"line_color": "purple"})



TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_d4496de5-a105-4f25-887c-c6c56beec6ea'.

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(8.9911746…

In [8]:
myCollector.collectAllPatches()

TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_c52a2ab9-a1b3-49df-8d85-8b4ee13a16f9'.

Start selecting patches
Patch 0/44694 selected!
Patch 1/44694 selected!
Patch 2/44694 selected!
Patch 3/44694 selected!
Patch 4/44694 selected!
Patch 5/44694 selected!
Patch 6/44694 selected!
Patch 7/44694 selected!
Patch 8/44694 selected!
Patch 9/44694 selected!
Patch 10/44694 selected!
Patch 11/44694 selected!
Patch 12/44694 selected!
Patch 13/44694 selected!
Patch 14/44694 selected!
Patch 15/44694 selected!
Patch 16/44694 selected!
Patch 17/44694 selected!
Patch 18/44694 selected!
Patch 19/44694 selected!
Patch 20/44694 selected!
Patch 21/44694 selected!
Patch 22/44694 selected!
Patch 23/44694 selected!
Patch 24/44694 selected!
Patch 25/44694 selected!
Patch 26/44694 selected!
Patch 27/44694 selected!
Patch 28/44694 selected!
Patch 29/44694 selected!
Patch 30/44694 selected!
Patch 31/44694 selected!
Patch 32/44694 selected!
Patch 33/44694 selected!
Patch 34/44694 selected!
Patch 35/44694 selected!
Patch 36/44694 selected!
Patch 37/44694 selected!
Patch 38/44694 selected!
Patch 39/44

 ...]